# Vesuvius Pretraining
Code to pretrain on Vesuvius unlabeled segments using Variance-Invariance-Covariance method with the lightly package for self-supervised training. 

based on this fantastic notebook https://www.kaggle.com/code/tanakar/2-5d-segmentaion-baseline-training

In [1]:
!pip install wandb pytorch-lightning lightly -qU

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.4.1 requires pydantic!=1.8,!=1.8.1,<1.10.0,>=1.7.4, but you have pydantic 1.10.12 which is incompatible.


In [8]:
!pip install timm warmup_scheduler segmentation_models_pytorch albumentations


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 72.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 96.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 38.8 MB/s eta 0:00:0000:0100:01
  Created wheel for warmup_scheduler: filename=warmup_scheduler-0.3-py3-none-any.whl size=2966 sha256=b44026ac2964358b9bad6fb126a54455a5996eaa3f5bb44c2ccf3d2003ad3d33
  Stored in directory: /root/.cache/pip/wheels/d1/b1/96/42c63380f9c322f577be6ba80cb7ab1bc1b28e10f74d95ba64
  Created wheel for efficientnet

In [2]:
import os.path as osp
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, StochasticWeightAveraging

import numpy as np
import pandas as pd

from pytorch_lightning.loggers import WandbLogger

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, log_loss
import pickle
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler
import warnings
import sys
import pandas as pd
import gc
import sys
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter
import cv2

import scipy as sp
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from functools import partial

import importlib

from torch.optim import Adam, SGD, AdamW
from ema_pytorch import EMA

import datetime
import math
import time
import numpy as np
import torch

from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
from warmup_scheduler import GradualWarmupScheduler
from torch.utils.data import DataLoader, Dataset
import cv2
import torch
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

In [4]:
# Log in to your W&B account
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
import segmentation_models_pytorch as smp

In [10]:
!pip install warmup_scheduler

## config

In [12]:
class CFG:
    # ============== comp exp name =============
    comp_name = 'vesuvius'

    # comp_dir_path = './'
    comp_dir_path = './'
    comp_folder_name = './'
    # comp_dataset_path = f'{comp_dir_path}datasets/{comp_folder_name}/'
    comp_dataset_path = f'./'
    
    exp_name = 'pretraining_30'

    # ============== pred target =============
    target_size = 1

    

    in_chans = 20 # 65
    encoder_depth=5
    # ============== training cfg =============
    size = 128
    tile_size = 128
    stride = tile_size // 4

    train_batch_size = 150 # 32
    valid_batch_size = train_batch_size*2
    use_amp = True

    scheduler = 'GradualWarmupSchedulerV2'
    # scheduler = 'CosineAnnealingLR'
    epochs = 35 # 30

    # adamW warmupあり
    warmup_factor = 10
    # lr = 1e-4 / warmup_factor
    lr = 1e-4 / warmup_factor


   
    min_lr = 1e-6
    weight_decay = 1e-6
    max_grad_norm = 5

    num_workers = 12

    seed = 42

    # ============== set dataset path =============
    print('set dataset path')

    outputs_path = f'./outputs/{comp_name}/{exp_name}/'

    submission_dir = outputs_path + 'submissions/'
    submission_path = submission_dir + f'submission_{exp_name}.csv'

    model_dir = outputs_path + \
        f'{comp_name}-models/'

    figures_dir = outputs_path + 'figures/'

    log_dir = outputs_path + 'logs/'
    log_path = log_dir + f'{exp_name}.txt'

    # ============== augmentation =============
    train_aug_list = [
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),

        A.ShiftScaleRotate(rotate_limit=45,shift_limit=0.15,scale_limit=0.15,p=0.5),
        A.OneOf([
                A.GaussNoise(var_limit=[10, 50]),
                A.GaussianBlur(),
                A.MotionBlur(),
                ], p=0.4),
        A.GridDistortion(num_steps=2, distort_limit=0.3, p=0.4),
        A.CoarseDropout(max_holes=4, max_width=int(size * 0.06), max_height=int(size * 0.06), 
                        mask_fill_value=0, p=0.5),
        A.Cutout(max_h_size=int(size * 0.15),
                 max_w_size=int(size * 0.15), num_holes=1, p=0.5),
        A.Normalize(
            mean= [0] * in_chans,
            std= [1] * in_chans
        ),
        ToTensorV2(transpose_mask=True),
        
    ]
    global_aug_list=[
        A.Resize(size, size),
        A.ShiftScaleRotate(rotate_limit=0,shift_limit=0.25,scale_limit=0.25,p=0.75),
    ]
    valid_aug_list = [
        A.Resize(size, size),
        A.Normalize(
            mean= [0] * in_chans,
            std= [1] * in_chans
        ),
        ToTensorV2(transpose_mask=True),
    ]


set dataset path


/usr/local/lib/python3.9/dist-packages/albumentations/augmentations/dropout/cutout.py:49: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  warnings.warn(


## helper

In [13]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [14]:
def init_logger(log_file):
    from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

def set_seed(seed=None, cudnn_deterministic=True):
    if seed is None:
        seed = 42

    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = cudnn_deterministic
    torch.backends.cudnn.benchmark = False

In [15]:
def make_dirs(cfg):
    for dir in [cfg.model_dir, cfg.figures_dir, cfg.submission_dir, cfg.log_dir]:
        os.makedirs(dir, exist_ok=True)

In [16]:
def cfg_init(cfg, mode='train'):
    set_seed(cfg.seed)
    if mode == 'train':
        make_dirs(cfg)

In [17]:
cfg_init(CFG)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## image, mask

In [18]:
from scipy import ndimage
def read_image_mask(fragment_id,start_idx=14,end_idx=36):

    images = []

    # idxs = range(65)
    mid = 65 // 2
    start = mid - CFG.in_chans // 2
    end = mid + CFG.in_chans // 2
    idxs = range(start_idx, end_idx)

    for i in tqdm(idxs):
        
        image = cv2.imread(CFG.comp_dataset_path + f"train/{fragment_id}/surface_volume/{i:02}.tif", 0)

        pad0 = (CFG.tile_size - image.shape[0] % CFG.tile_size)
        pad1 = (CFG.tile_size - image.shape[1] % CFG.tile_size)

        image = np.pad(image, [(0, pad0), (0, pad1)], constant_values=0)
        
        image = cv2.resize(image, (image.shape[1]//2,image.shape[0]//2), interpolation = cv2.INTER_AREA)
        images.append(image)
    images = np.stack(images, axis=2)
    mask = cv2.imread(CFG.comp_dataset_path + f"train/{fragment_id}/inklabels.png", 0)
    mask = np.pad(mask, [(0, pad0), (0, pad1)], constant_values=0)
    fragment_mask=cv2.imread(CFG.comp_dataset_path + f"train/{fragment_id}/mask.png", 0)
    fragment_mask = np.pad(fragment_mask, [(0, pad0), (0, pad1)], constant_values=0)
    kernel = np.ones((16,16),np.uint8)
    fragment_mask = cv2.erode(fragment_mask,kernel,iterations = 1)
    mask = mask.astype('float32')
    mask = cv2.resize(mask, (mask.shape[1]//2,mask.shape[0]//2), interpolation = cv2.INTER_AREA)
    fragment_mask = cv2.resize(fragment_mask, (fragment_mask.shape[1]//2,fragment_mask.shape[0]//2), interpolation = cv2.INTER_AREA)

    mask/=255
    return images, mask,fragment_mask
from scipy import ndimage
def read_image_mask_scrolls(fragment_id,start_idx=14,end_idx=36):

    images = []

    # idxs = range(65)
    mid = 65 // 2
    start = mid - CFG.in_chans // 2
    end = mid + CFG.in_chans // 2
    idxs = range(start_idx, end_idx)

    for i in idxs:
        
        image = cv2.imread(f"./scrolls_all/{fragment_id}/layers/{i:02}.tif", 0)

        pad0 = (CFG.tile_size - image.shape[0] % CFG.tile_size)
        pad1 = (CFG.tile_size - image.shape[1] % CFG.tile_size)

        image = np.pad(image, [(0, pad0), (0, pad1)], constant_values=0)
        images.append(image)
    images = np.stack(images, axis=2)
    fragment_mask=None
    if os.path.exists(f'./scrolls_all/{fragment_id}/{fragment_id}_mask.png'):
        fragment_mask=cv2.imread(CFG.comp_dataset_path + f"scrolls_all/{fragment_id}/{fragment_id}_mask.png", 0)
        fragment_mask = np.pad(fragment_mask, [(0, pad0), (0, pad1)], constant_values=0)
        kernel = np.ones((16,16),np.uint8)
        fragment_mask = cv2.erode(fragment_mask,kernel,iterations = 1)
        # fragment_mask = cv2.rotate(fragment_mask, cv2.ROTATE_90_CLOCKWISE)

    # mask = 2*(mask/255 - 0.5)
    return images,fragment_mask
def read_image_scrolls_stats(fragment_id,start_idx=16,end_idx=32):

    images = []

    # idxs = range(65)
    mid = 65 // 2
    start = mid - 16 // 2
    end = mid + 16// 2
    idxs = range(start_idx, end_idx)
    means=[]
    stds=[]
    for i in idxs:
        
        image = cv2.imread(f"./scrolls_all/{fragment_id}/layers/{i:02}.tif", 0)
        pad0 = (64- image.shape[0] % 64)
        pad1 = (64- image.shape[1] %64)

        image = np.pad(image, [(0, pad0), (0, pad1)], constant_values=0).astype(np.uint8)
        means.append(image.mean())
        stds.append(image.std())
    return np.array(means),np.array(stds)

In [19]:
def get_img_splits(fragment_id,s,e):
    images = []
    masks = []
    xyxys = []
    image, mask,fragment_mask = read_image_mask(fragment_id,s,e)
    x1_list = list(range(0, image.shape[1]-CFG.tile_size+1, CFG.stride))
    y1_list = list(range(0, image.shape[0]-CFG.tile_size+1, CFG.stride))
    for y1 in y1_list:
        for x1 in x1_list:
            y2 = y1 + CFG.tile_size
            x2 = x1 + CFG.tile_size
            images.append(image[y1:y2, x1:x2])
            masks.append(mask[y1:y2, x1:x2, None])
            xyxys.append([x1, y1, x2, y2])
    test_dataset = CustomDatasetTest(images,np.stack(xyxys), CFG, transform=get_transforms(data='valid', cfg=CFG))

    test_loader = DataLoader(test_dataset,
                              batch_size=CFG.valid_batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False,
                              )
    return test_loader, np.stack(xyxys),mask.shape

In [20]:
def get_train_valid_dataset(frags):
    train_images = []
    train_masks = []

    valid_images = []
    valid_masks = []
    valid_xyxys = []

    for fragment_id in range(4, 5):

        image, _,fragment_mask = read_image_mask(fragment_id)

        x1_list = list(range(0, image.shape[1]-CFG.tile_size+1, CFG.stride))
        y1_list = list(range(0, image.shape[0]-CFG.tile_size+1, CFG.stride))

        for y1 in y1_list:
            for x1 in x1_list:
                y2 = y1 + CFG.tile_size
                x2 = x1 + CFG.tile_size

                if not np.any(fragment_mask[y1:y2, x1:x2]==0) :
                    train_images.append(image[y1:y2, x1:x2])

    for fragment_id in tqdm(frags):
        image,fragment_mask = read_image_mask_scrolls(fragment_id)
        x1_list = list(range(0, image.shape[1]-CFG.tile_size+1, CFG.stride))
        y1_list = list(range(0, image.shape[0]-CFG.tile_size+1, CFG.stride))

        for y1 in y1_list:
            for x1 in x1_list:
                y2 = y1 + CFG.tile_size
                x2 = x1 + CFG.tile_size
        
                if fragment_mask is not None and not np.any(fragment_mask[y1:y2, x1:x2]==0) :
                    train_images.append(image[y1:y2, x1:x2])
                elif not np.all(image[y1:y2, x1:x2])==0:
                    train_images.append(image[y1:y2, x1:x2])

    return train_images

In [21]:
train_images = get_train_valid_dataset(os.listdir('./scrolls_all'))

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

In [22]:
import gc
gc.collect()

37

## dataset

In [25]:
import numpy as np
from torch.utils.data import DataLoader, Dataset
import cv2
import torch
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

In [26]:
def get_transforms(data, cfg):
    if data == 'train':
        aug = A.Compose(cfg.train_aug_list)
    elif data == 'valid':
        aug = A.Compose(cfg.global_aug_list)

    # print(aug)
    return aug

class CustomDatasetPretraining(Dataset):
    def __init__(self, images, cfg, transform=None,global_transform=None):
        self.images = images
        self.cfg = cfg
        self.transform = transform
        self.global_transform=global_transform
    def __len__(self):
        # return len(self.df)
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        offset=random.choice([0,1,1,2])
        # offset=1
        stretch_offset=random.choice([0,1,1,2,3,4])
        pixel_offset=0
        pixel_offset=random.choice([0,0,0,5,10])
        image=image[:,:,offset:offset+self.cfg.in_chans]+pixel_offset
        if self.transform:
            data = self.transform(image=image)
            image1 = data['image'].unsqueeze(0)
            data2= self.transform(image=image)
            image2= data2['image'].unsqueeze(0)
        return image1,image2

In [27]:

train_dataset = CustomDatasetPretraining(
    train_images, CFG, transform=get_transforms(data='train', cfg=CFG),global_transform=get_transforms(data='valid', cfg=CFG))

train_loader = DataLoader(train_dataset,
                          batch_size=CFG.train_batch_size,
                          shuffle=True,
                          num_workers=CFG.num_workers, pin_memory=True, drop_last=True,
                          )

## model

In [29]:
from resnetall import generate_model
# from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules import SimSiamPredictionHead, SimSiamProjectionHead
from lightly.utils.debug import std_of_l2_normalized
import copy

from lightly.loss.vicreg_loss import VICRegLoss
from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules import BYOLPredictionHead, BYOLProjectionHead
from lightly.models.utils import deactivate_requires_grad, update_momentum
from lightly.transforms.simclr_transform import SimCLRTransform
from lightly.utils.scheduler import cosine_schedule

## The projection head is the same as the Barlow Twins one
from lightly.models.modules import BarlowTwinsProjectionHead
def normalization(x):
    """input.shape=(batch,f1,f2,...)"""
    #[batch,f1,f2]->dim[1,2]
    dim=list(range(1,x.ndim))
    mean=x.mean(dim=dim,keepdim=True)
    std=x.std(dim=dim,keepdim=True)
    return (x-mean)/(std+1e-9)
class BYOL(pl.LightningModule):
    def __init__(self,model_depth=50):
        super().__init__()
        self.save_hyperparameters()
        self.backbone=generate_model(model_depth=self.hparams.model_depth, n_input_channels=1,n_classes=512,no_max_pool=True)
        self.projection_head = BYOLProjectionHead(512, 1024, 256)
        self.prediction_head = BYOLPredictionHead(256, 1024, 256)

        self.backbone_momentum = copy.deepcopy(self.backbone)
        self.projection_head_momentum = copy.deepcopy(self.projection_head)

        deactivate_requires_grad(self.backbone_momentum)
        deactivate_requires_grad(self.projection_head_momentum)

        self.criterion = NegativeCosineSimilarity()

    def forward(self, x):
        y = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(y)
        p = self.prediction_head(z)
        return p

    def forward_momentum(self, x):
        y = self.backbone_momentum(x).flatten(start_dim=1)
        z = self.projection_head_momentum(y)
        z = z.detach()
        return z

    def training_step(self, batch, batch_idx):
        momentum = cosine_schedule(self.current_epoch, 10, 0.996, 1)
        update_momentum(self.backbone, self.backbone_momentum, m=momentum)
        update_momentum(self.projection_head, self.projection_head_momentum, m=momentum)
        x0, x1 = batch
        p0 = self.forward(x0)
        z0 = self.forward_momentum(x0)
        p1 = self.forward(x1)
        z1 = self.forward_momentum(x1)
        loss = 0.5 * (self.criterion(p0, z1) + self.criterion(p1, z0))
        std_rep=std_of_l2_normalized(p0)

        self.log("train/NegativeCS", loss,on_step=True, on_epoch=True, prog_bar=True)
        self.log("train/STD_of_repr", std_rep,on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer=torch.optim.SGD(
            self.parameters(),
            lr=0.005,
            momentum=0.9,
            weight_decay=2e-6,
            nesterov=True,
        ) 

        scheduler = get_scheduler(CFG, optimizer)
        return [optimizer],[scheduler]    
class VICREGPLModel(pl.LightningModule):
    def __init__(self,model_depth=50):
        super(RegressionPLModel, self).__init__()

        self.save_hyperparameters()
        self.backbone=generate_model(model_depth=self.hparams.model_depth, n_input_channels=1,n_classes=512,forward_features=False)
        self.projection_head = BarlowTwinsProjectionHead(512, 256, 512)
        self.criterion = VICRegLoss(nu_param=5.0)
        self.normalization=nn.InstanceNorm3d(num_features=1)
    def forward(self, x):
        x=self.normalization(x)
        x = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(x)
        return z

    def training_step(self, batch, batch_idx):
        x0, x1 = batch
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        std_rep=std_of_l2_normalized(z0)

        self.log("train/VICREGloss", loss,on_step=True, on_epoch=True, prog_bar=True)
        self.log("train/STD_of_repr", std_rep,on_step=True, on_epoch=True, prog_bar=True)

        return loss


    def configure_optimizers(self):

        optimizer=torch.optim.SGD(
            self.parameters(),
            lr=0.05,
            momentum=0.9,
            weight_decay=5e-5,
            nesterov=True,
        ) 

        scheduler = get_scheduler(CFG, optimizer)
        return [optimizer],[scheduler]

class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    """
    https://www.kaggle.com/code/underwearfitting/single-fold-training-of-resnet200d-lb0-965
    """
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(
            optimizer, multiplier, total_epoch, after_scheduler)

    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [
                        base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

def get_scheduler(cfg, optimizer):
    scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, cfg.epochs, eta_min=1e-4)
    scheduler = GradualWarmupSchedulerV2(
        optimizer, multiplier=10, total_epoch=1, after_scheduler=scheduler_cosine)

    return scheduler

def scheduler_step(scheduler, avg_val_loss, epoch):
    scheduler.step(epoch)
   


In [ ]:
# torch.multiprocessing.set_start_method('spawn')
for di,depth in enumerate([50]):
    torch.set_float32_matmul_precision('medium')
    wandb_logger = WandbLogger(project="vesivus",name=f'pretraining_{depth}')
    model=VICREGPLModel(model_depth=depth)
    # wandb_logger.watch(model, log="all", log_freq=200)
    trainer = pl.Trainer(
        max_epochs=35,
        accelerator="gpu",
        # strategy='auto',
        devices=1,
        logger=wandb_logger,
        default_root_dir="./models",
        accumulate_grad_batches=2,
        precision='16-mixed',
        gradient_clip_val=1.0,
        gradient_clip_algorithm="norm",
        callbacks=[ModelCheckpoint(filename=f'round2_{depth}'+'{epoch}',dirpath=CFG.model_dir,monitor='train/VICREGloss',save_top_k=50,mode='min',every_n_epochs=1),
                  StochasticWeightAveraging(swa_lrs=1e-2)]
    )
    trainer.fit(model=model, train_dataloaders=train_loader)
    #ckpt_path=CFG.model_dir+'round2_50epoch=9.ckpt'
    wandb.finish()

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
wandb: Currently logged in as: joenader. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.9/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /notebooks/outputs/vesuvius/pretraining_30/vesuvius-models exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                      | Params
--------------------------------------------------------------
0 | backbone        | ResNet                    | 47.2 M
1 | projection_head | BarlowTwinsProjectionHead | 329 K 
2 | criterion       | VICRegLoss                | 0     
3 | normalization   | InstanceNorm3d            | 0     
--------------------------------------------------------------
47.5 M    Trainable params
0         Non-trainable params
47.5 M    Total params
190.134   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/torch/optim/lr_scheduler.py:728: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


### Code for plotting CKA similarity between the models representation

In [32]:
# ckpt_dirs=[
# './outputs/vesuvius/pretraining_28/vesuvius-models/round2_50epoch=6.ckpt',
# './outputs/vesuvius/pretraining_28/vesuvius-models/round2_50epoch=12.ckpt',
# './outputs/vesuvius/pretraining_28/vesuvius-models/round2_50epoch=15.ckpt',
# './outputs/vesuvius/pretraining_28/vesuvius-models/round2_50epoch=17.ckpt',
# './outputs/vesuvius/pretraining_28/vesuvius-models/round2_50epoch=20.ckpt',
# './outputs/vesuvius/pretraining_28/vesuvius-models/round2_50epoch=23.ckpt',
# ]
# from cka_utils import *
# from tqdm.notebook import tqdm

# # ckpt_dirs=sorted([os.path.join(CFG.model_dir,model_path) for model_path in os.listdir(CFG.model_dir) if '.ckpt' in model_path])
# models=[RegressionPLModel.load_from_checkpoint(path) for path in ckpt_dirs]
# models=[m.eval().cuda() for m in models]
# n_models=len(models)

# embeddings=np.zeros((n_models,512,512))
# for i in tqdm(range(512)):
#     x1,x2=train_dataset[i]
#     x=x1.cuda().unsqueeze(0)
#     for mi,model in enumerate(models):
#         embeddings[mi,i,:]=model(x).detach().cpu().numpy()
# import seaborn as sns
# ckas=np.zeros((n_models,n_models))
# for i in range(n_models):
#     for j in range(n_models):
#         if i==j:
#             continue
#         ckas[i,j]=cka(gram_rbf(embeddings[i], 0.5), gram_rbf(embeddings[j], 0.5))
# ckas_lin=np.zeros((n_models,n_models))
# for i in range(n_models):
#     for j in range(n_models):
#         if i==j:
#             continue
#         ckas_lin[i,j]=feature_space_linear_cka(embeddings[i],embeddings[j])


In [31]:
# fig,ax=plt.subplots(1,2,figsize=(20,10))
# sns.heatmap(ckas_lin,annot=True,ax=ax[1])
# sns.heatmap(ckas,annot=True,ax=ax[0])
# plt.show()
# for model_path,i in zip(ckpt_dirs,range(n_models)):
#     print(model_path,std_of_l2_normalized(torch.Tensor(embeddings[i])))